In [1]:
# !git commit -m "prediction age is completed 90%"

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from keras.utils import np_utils

In [37]:
dataset = pd.read_csv('dataset_200.csv')[500:1500]

In [38]:
s = str(dataset['image'].values[0])
print(s)

../../age_detection/age_gender_race/53_1_1_20170113001807851.jpg.chip.jpg


In [39]:
gray = cv2.cvtColor(cv2.imread(s), cv2.COLOR_BGR2GRAY)
gray.shape

(200, 200)

In [40]:
X = dataset.drop(columns=['target_age','target_gender','target_race','age','Unnamed: 0'])
y = dataset.drop(columns=['image','target_gender','target_race','age','Unnamed: 0'])
X

,image
500,../../age_detection/age_gender_race/53_1_1_201...
501,../../age_detection/age_gender_race/65_1_0_201...
502,../../age_detection/age_gender_race/29_0_2_201...
503,../../age_detection/age_gender_race/26_0_0_201...
504,../../age_detection/age_gender_race/39_1_1_201...
...,...
1495,../../age_detection/age_gender_race/55_0_0_201...
1496,../../age_detection/age_gender_race/1_0_2_2016...
1497,../../age_detection/age_gender_race/35_1_2_201...
1498,../../age_detection/age_gender_race/42_1_4_201...


In [41]:
print(X.shape,y.shape)

(1000, 1) (1000, 1)


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [43]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [44]:
from tqdm import tqdm

In [45]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['image']):
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face =cv2.resize(face, (200,200))
        face = np.expand_dims(face, axis=2)
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

In [46]:
X_train = get_data_from_local(X_train)
X_test = get_data_from_local(X_test)

100%|██████████| 50/50 [00:00<00:00, 142.25it/s]


In [47]:
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
print(y_train.shape,y_test.shape,X_train.shape,X_test.shape)

(950, 7) (50, 7) (950, 200, 200, 1) (50, 200, 200, 1)


In [15]:
final_cnn = Sequential()
final_cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))  
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(GlobalAveragePooling2D())

final_cnn.add(Dense(132, activation='relu'))

final_cnn.add(Dense(7, activation='softmax'))

final_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
average_pooling2d_2 (Average (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 256)       2

In [16]:
final_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
#load the h5

In [22]:
# Creating a ModelCheckpoint callback object to save the model according to the value of val_accuracy.

checkpoint = ModelCheckpoint(filepath=f"final_cnn_model_checkpoint.tf",
                             monitor='val_accuracy',
                             save_format='tf',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )

In [48]:
from keras.models import load_model
weights_path = 'final_cnn_model_checkpoint.tf'
final_cnn = load_model(weights_path)

In [49]:
# Fitting the above created CNN model.

final_cnn_history = final_cnn.fit(x=X_train,
                                  y=y_train,
                                  batch_size=62,
                                  validation_data=(X_test,y_test),
                                  epochs=15,
                                  callbacks=[checkpoint],
                                  shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                                 )

Epoch 1/15
16/16 [==============================] - ETA: 0s - loss: 1.8039 - accuracy: 0.3189
Epoch 00001: val_accuracy did not improve from 0.52000
16/16 [==============================] - 39s 2s/step - loss: 1.8039 - accuracy: 0.3189 - val_loss: 1.7410 - val_accuracy: 0.3200
Epoch 2/15
16/16 [==============================] - ETA: 0s - loss: 1.7775 - accuracy: 0.3189
Epoch 00002: val_accuracy did not improve from 0.52000
16/16 [==============================] - 42s 3s/step - loss: 1.7775 - accuracy: 0.3189 - val_loss: 1.7198 - val_accuracy: 0.3200
Epoch 3/15
16/16 [==============================] - ETA: 0s - loss: 1.7508 - accuracy: 0.3189
Epoch 00003: val_accuracy did not improve from 0.52000
16/16 [==============================] - 44s 3s/step - loss: 1.7508 - accuracy: 0.3189 - val_loss: 1.6968 - val_accuracy: 0.3200
Epoch 4/15
16/16 [==============================] - ETA: 0s - loss: 1.7289 - accuracy: 0.3189
Epoch 00004: val_accuracy did not improve from 0.52000
16/16 [==========